# Task 1.1

In [ ]:
import numpy as np

In [ ]:
N = 64
r = 0.5
K = 16

In [ ]:
def f(x,y):
    partial_sum = 0
    a_ij = np.random.normal(0, 1, K*K)
    for i in range(K):
        for j in range(K):
            partial_sum += a_ij * (i**2 + j**2)**r * np.sin(np.pi * i * x) * np.sin(np.pi * j * y)

    return np.pi  / K^2